<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h2><font color= "blue" size="+3">PyCon 2024 Tutorial</font></h2>
</center>

---

<center>
    <h3>Python Workflows to Extract and Plot Satellite Data Products along Tracks</h3>
    <h2><font color="red" size="+3">International Space Station - Option 2</font></h2>
</center>

----
[Jules Kouatchou](mailto:Jules.Kouatchou@nasa.gov) • [Bruce Van Aartsen](mailto:bruce.vanaartsen@nasa.gov)
-
----

## <font color="blue">Objective</font>

- The International Space Station (ISS) makes 16 orbits of Earth, traveling through 16 sunrises and sunsets.
- ISS travels at a speed of five miles per second, circling Earth about every 90 minutes.
- We wrote a Python code that collects every 5 seconds (for over 5 hours) the position (latitude/longitude) and the date/time.
- For each ISS position, the code determines:
   - If it is a land or not (ocean)
   - The name of the country
   - The weather condition (temperature and wind speed)
   - The track id. A new track is set when ISS crosses the International Date Line (180th meridian)
- The collected time series data is written in a CSV file.

__The goal of this exercise is to read the CSV file using Pandas and then use GeoPandas and MovingPandas to perform data manipulation and visualization.__

To have more information on this exercise and to see what kind of manipulations we can do with the data, please check:

[Python Tool for Tracking the Movement of the International Space Station and Identifying the Weather Condition along the Track](https://github.com/astg606/pycon/tree/main/year2024/poster#readme)

## <font color="blue">Import necessary modules</font>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import datetime as dt
from pathlib import Path

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
import movingpandas as mpd

In [ ]:
import holoviews as hv
import hvplot.pandas 

In [ ]:
figsize = (17, 11)

## <font color="blue">Read the CSV file with Pandas</font>

- The CSV file has eight columns:
   - `t`: for the date/time as strings
   - `latitude` as floats
   - `longitude` as floats
   - `land_flag`
   - `temperature`
   - `windspeed`
   - `traj_id`: identify an orbit
   - `country`: country of each location
- While reading the file, make sure that you transform the `t` values into datetime objects.
- Rename the columns.
- Create a new `geometry` columns

In [ ]:
#data_dir = "/Users/jkouatch/myTasks/PythonTraining/ASTG606/Materials/sat_data/ISS_Data/"
data_dir = "/tljh-data/sat_data/ISS_Data"

In [ ]:
file_name = Path(data_dir) / "iss_timeseries_trajectories_20240303_104539.csv"

#### <font color="green">Read the file to create a Pandas DataFrame</font>

Do no forget to handle the date/time.

In [ ]:
dateparse = lambda x: ...
df_iss = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
dateparse = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df_iss = pd.read_csv(file_name,
                parse_dates={'datetime': [0,]}, 
                date_parser=dateparse)
```
</details>

#### <font color="green">Rename the `datetime` column to `t`</font>

In [ ]:
df_iss.

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
df_iss.rename(columns = {'datetime': 't'}, inplace=True)
```
</details>

#### <font color="green">Add a `geometry` column</font>

In [ ]:
df_iss['geometry'] = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
df_iss['geometry'] = [shpgeom.Point(xy) for xy in zip(df_iss['longitude'], df_iss['latitude'])] 
```
</details>

## <font color="blue">Manipulation with GeoPandas</font>

#### <font color="green">Use GeoPandas to create the GeoDataFrame</font>

In [1]:
gdf_iss = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
gdf_iss = gpd.GeoDataFrame(df_iss, geometry="geometry") 
```
</details>

#### Interactive plot of the path of ISS

In [ ]:
gdf_iss.hvplot(tiles='EsriTerrain', coastline=True, 
           hover_cols=["t"])

## <font color="blue">Manipulation with MovingPandas</font>

#### <font color="green">Create the MovingPandas trajectory</font>

__Note that we have here an identifier for the orbits.__

In [ ]:
mdf_iss = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
mdf_iss = mpd.Trajectory(df_iss, traj_id_col="traj_id",
                      x = "longitude", y="latitude", t="t")
```
</details>

#### Basic plot of the trajectory and the buffer

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
mdf_iss.plot(ax=ax);

#### Interactive plot of the trajectory

In [ ]:
mdf_iss.hvplot(line_width=0.9)

## <font color="blue">Perform analyses</font>